# Triplet network

# Imports

In [ ]:
# # # Credit: https://stackoverflow.com/questions/34199233/how-to-prevent-tensorflow-from-allocating-the-totality-of-a-gpu-memory
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
from tensorflow import keras
from loaders import TripletSequence
from loaders import PairSequence
import data
import nets

# Globals

In [ ]:
K = keras.backend
codings_size = 10
alpha = 0.5

# Data

In [ ]:
emnist = data.load_dataset('emnist/balanced')
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = emnist.split()

In [ ]:
train_seq = TripletSequence(train_x, train_y, samples_per_label=1, batch_size=1024, can_shuffle=True, is_generator=True)
valid_seq = TripletSequence(valid_x, valid_y, samples_per_label=1, batch_size=1024, can_shuffle=True, is_generator=True)
test_seq  = TripletSequence(test_x,  test_y,  samples_per_label=1, batch_size=1024, can_shuffle=True, is_generator=True)

### Make the triplet model

In [ ]:
conv_base   = nets.make_conv_base(codings_size)
triplet_net = nets.TripletNet(conv_base)

In [ ]:
my_triplet_loss = nets.get_triplet_loss(alpha)

In [ ]:
opt = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=True)

In [ ]:
triplet_net.compile(loss=my_triplet_loss, 
                    metrics=nets.TripletAccuracy(alpha),
                    optimizer=opt, 
                    run_eagerly=True
                   )

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(patience=20, min_delta=1/100000, restore_best_weights=True, monitor='val_loss')

In [ ]:
triplet_net.fit(train_seq, epochs=500, validation_data=valid_seq, workers=26, callbacks=[early_stopping])

# Evaluation

In [ ]:
print("Evaluating training set:")
for i in range(3):
    print(triplet_net.evaluate(train_seq, verbose = 0)[1])

In [ ]:
print("Evaluating validation set:")
for i in range(3):
    print(triplet_net.evaluate(valid_seq, verbose = 0)[1])

In [ ]:
print("Evaluating testing set:")
for i in range(3):
    print(triplet_net.evaluate(test_seq, verbose = 0)[1])

In [ ]:
print("Saving Model")
triplet_net.save("tn_model")